<a href="https://colab.research.google.com/github/alexa21e/cifar10-image-classification/blob/master/Deep_Learning_CIFAR10_image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import os
import sys

DRIVE_PROJECT_ROOT = '/content/drive/MyDrive/Colab Notebooks'

drive.mount('/content/drive')

os.chdir(DRIVE_PROJECT_ROOT)
!pip install -r required_packages.txt

SRC_PATH = os.path.join(DRIVE_PROJECT_ROOT, 'src')
if SRC_PATH not in sys.path:
    sys.path.insert(0, SRC_PATH)

print("Finalized installing stage")

from train_utils import load_and_preprocess_cifar10, create_baseline_cnn

(features_train, labels_train), (features_test, labels_test) = load_and_preprocess_cifar10()

baseline_model = create_baseline_cnn()

baseline_model.summary()

print("Finalized loading stage")

In [ ]:
history = baseline_model.fit(
    # ARGUMENT 1: the input data (the images) the model will learn patterns from
    features_train,

    # ARGUMENT 2: the expected output (labels) the model aims to predict correctly
    labels_train,

    # ARGUMENT 3: epochs are the number of times the model will iterate over the
    # training dataset, repetitiveness  usually leads to better learning
    epochs=20,

    # ARGUMENT 4: batch_size is the number of samples processed before the model's
    # internal weights are updated, a larger batch size uses the GPU memory efficiently
    batch_size=128,

    # ARGUMENT 5: provides the model with a separate dataset made of both images
    # and labels to evaluate its performance after each epoch
    # this step can detect overfitting which appears when validation accuracy
    # stops increasing
    validation_data=(features_test, labels_test),

    # ARGUMENT 6: controls the amount of logs during training, has values 0, 1 or 2
    verbose=1
)

print("Finalized training")

loss, acc = baseline_model.evaluate(features_test, labels_test, verbose=0)
print(f"Accuracy on test training set: {acc*100:.2f}%")

drive_save_path = os.path.join(DRIVE_PROJECT_ROOT, 'baseline_cnn_weights.keras')
baseline_model.save(drive_save_path)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
history_df = pd.DataFrame(history.history)

plt.figure(figsize=(10, 6))
plt.plot(history_df['accuracy'], label='Training Accuracy')
plt.plot(history_df['val_accuracy'], linestyle='--', label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()

plt.figure(figsize=(10, 6))
plt.plot(history_df['loss'], label='Training Loss')
plt.plot(history_df['val_loss'], linestyle='--', label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()